In [2]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import f1_score, precision_score, recall_score
import joblib
import pandas as pd
import numpy as np

In [3]:
dataset = pd.read_csv('data/winequality-red.csv', sep=';')

In [4]:
dataset.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [6]:
dataset.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


Переведём целевую переменную в бинарное значение.

In [7]:
dataset['quality'] = dataset['quality'].apply(lambda x: 0 if x<6.5 else 1)

In [8]:
#разделение со стратификацией

def streaming_reading(df, splits=3):
    X = []
    y = []
    current_line = 0
    df = shuffle(df, random_state=0)
    df0 = df[df.quality==0]
    df1 = df[df.quality==1]
    proporcion = df1.shape[0]/df0.shape[0]
    X_train0 = df0.drop('quality', axis = 1)
    X_train1 = df1.drop('quality', axis = 1)
    y_train0 = df0['quality']
    y_train1 = df1['quality']
    train_data0 = X_train0.to_numpy()
    train_data1 = X_train1.to_numpy()
    
    for i in range(0, splits):
        len0 = round(len(train_data0)/splits)
        len1 = round(len0*proporcion)
        start0 = round(len0*i)
        start1 = round(len1*i)
        X.extend(train_data0[start0:(start0+len0)])
        X.extend(train_data1[start1:(start1+len1)])
        y.extend(y_train0[start0:(start0+len0)])
        y.extend(y_train1[start1:(start1+len1)])

        X, y = np.array(X), np.array(y)
        yield X, y
        X, y = [], []
        current_line = 0

In [9]:
def IncrementaLightGbm(X, y):  
    gbm = None

    params = {
        'task': 'train',
        'application': 'binary',  
        'boosting_type': 'gbdt', 
        'learning_rate': 0.05,  
        'tree_learner': 'serial',
        'metric': ['binary_logloss', 'auc'], 
        'max_bin': 255,
    }
    streaming_train_iterators = streaming_reading(dataset, splits=3)

    for i, data in enumerate(streaming_train_iterators):
        X_batch = data[0]
        y_batch = data[1]
        X_train, X_test, y_train, y_test = train_test_split(X_batch, y_batch, stratify=y_batch, test_size=0.1, random_state=0)
        y_train = y_train.ravel()
        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

        gbm = lgb.train(params,
                        lgb_train,
                        num_boost_round=1000,
                        valid_sets=lgb_eval,
                        init_model=gbm, 
                        keep_training_booster=True)  

        print("{} time".format(i))  
        score_train = dict([(score[1], score[2]) for score in gbm.eval_train()])
        print('The score of the current model in the training set is: logloss=%.4f, auc=%.4f, \n'
              % (score_train['binary_logloss'], score_train['auc']))

    return gbm

In [10]:
train_X, test_X, train_y, test_y = train_test_split(dataset.drop('quality', axis = 1),
                                                    dataset['quality'],
                                                    stratify=dataset['quality'],
                                                    test_size = 0.2)
gbm = IncrementaLightGbm(train_X, train_y)
pred_y = gbm.predict(test_X)
pred_classes = np.where(pred_y > 0.5, 1, 0)

[LightGBM] [Info] Number of positive: 65, number of negative: 414
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007783 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 690
[LightGBM] [Info] Number of data points in the train set: 479, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.135699 -> initscore=-1.851479
[LightGBM] [Info] Start training from score -1.851479
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

In [11]:
print(f'F1 score: {f1_score(test_y, pred_classes)}')
print('------------------------------------------')
print(f'Precision: {precision_score(test_y, pred_classes)}')
print('------------------------------------------')
print(f'Recall: {recall_score(test_y, pred_classes)}')

joblib.dump(gbm, 'loan_model.pkl')
gbm = joblib.load('loan_model.pkl')

F1 score: 0.8636363636363636
------------------------------------------
Precision: 0.8444444444444444
------------------------------------------
Recall: 0.8837209302325582


Метрики уже вроде нормальные, добавила в разделение стратификацию.

In [12]:
from prometheus_client import Gauge, start_http_server
import random
import time

In [13]:
g1 = Gauge('F1_score', 'Метрика модели')
g2 = Gauge('Precision', 'Метрика модели')
g3 = Gauge('Recall', 'Метрика модели')

In [14]:
g1.set(round(f1_score(test_y, pred_classes), 3))
g2.set(round(precision_score(test_y, pred_classes), 3))
g3.set(round(recall_score(test_y, pred_classes), 3))
start_http_server(8000)

(<prometheus_client.exposition.start_wsgi_server.<locals>.TmpServer at 0x7e1b8eefb5f0>,
 <Thread(Thread-4 (serve_forever), started daemon 138656767018688)>)